# Classification de textes : l'exemple des commentaires

In [ ]:
# Installations librairies tiers
!pip install spacy-langdetect

     |████████████████████████████████| 998 kB 5.7 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.7-py3-none-any.whl size=993431 sha256=2f4d9f0a7f2cedad1ede09d3b18f79b0f75bd131f35d8fb5988f2b57dd1b34cc
  Stored in directory: /root/.cache/pip/wheels/89/79/3b/9885ae7f4308f73c514f96d8574d40d7d8173a27731b674013
Successfully built langdetect


In [ ]:
# ainsi que le modèle moyen français de Spacy
!pip install https://github.com/explosion/spacy-models/releases/download/fr_core_news_md-3.2.0/fr_core_news_md-3.2.0.tar.gz
!pip install fr_core_news_md

     |████████████████████████████████| 46.9 MB 1.7 MB/s 
     |████████████████████████████████| 6.0 MB 5.1 MB/s 
     |████████████████████████████████| 181 kB 38.0 MB/s 
     |████████████████████████████████| 451 kB 26.4 MB/s 
     |████████████████████████████████| 628 kB 37.0 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 10.1 MB 27.3 MB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-3.2.0-py3-none-any.whl size=46938366 sha256=1517ef85c432e1b000e074989c457a7c76e6ef62a3d152a92d1bedc68b9666f1
  Stored in directory: /root/.cache/pip/wheels/ba/71/98/53c0dd09503b9eefd53afb0b60998dba3b2e41759e0b1b41ce
Successfully built fr-core-news-md
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successf

In [ ]:
# A installer pour utiliser concaténation de Doc2Vec
!pip install testfixtures

     |████████████████████████████████| 95 kB 1.4 MB/s 


##### Installation des librairies HuggingFace non installées par défaut dans Colab

In [1]:
!pip install datasets

     |████████████████████████████████| 311 kB 11.1 MB/s 
     |████████████████████████████████| 133 kB 47.6 MB/s 
     |████████████████████████████████| 1.1 MB 31.0 MB/s 
     |████████████████████████████████| 243 kB 49.1 MB/s 
     |████████████████████████████████| 67 kB 4.9 MB/s 
     |████████████████████████████████| 94 kB 2.9 MB/s 
     |████████████████████████████████| 271 kB 49.8 MB/s 
     |████████████████████████████████| 144 kB 40.5 MB/s 


In [2]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 11.8 MB/s 
     |████████████████████████████████| 596 kB 37.8 MB/s 
     |████████████████████████████████| 6.8 MB 37.4 MB/s 
     |████████████████████████████████| 895 kB 44.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## ICI IL FAUT REDEMARRER L'ENVIRONNEMENT D'EXECUTION

In [ ]:
# 1.a
# librairies générales
import os
import csv
import re
import datetime as dt
import pickle
import math
# librairies générales data science
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
# 1.b
# pour affichage dtypes quand plus de 60 colonnes
pd.set_option('display.max_rows', 240)
# pour affichage des bars tqdm avec apply et map
tqdm.pandas()

In [ ]:
# 1.c
# la librairie graphique la plus simple d'utilisation
#import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# 1.d imports spécifiques à google colab
from google.colab import files, drive

In [ ]:
# 1.e passage obligé pour récupérer un fichier sur le drive
# il va falloir choisir le drive (account Google) puis renseigner un mot de passe généré
drive.mount('/content/drive')

Mounted at /content/drive


#### Les autres imports vont se faire au fur et à mesure des besoins

## Phase préliminaire de constitution du fichier labellisé (avec des étiquettes représentant des "macro-émotions"

### On crée une initialisation des émotions à partir :<p>
- des emojis (dont les plus courant ont été labellisés à la main sur 10 émotions un peu arbitraires)
- des scores intrinsèques aux mots (lemmes), via le dictionnaire FEEL.csv des émotions du lexique français
Ce lexique fait référence à la polarité (positive / negative), et aux 6 émotions classiques (finalement peu intéressantes d'ailleurs)<p>
Un autre dictionnaire exploitable pourrait être le dictionnaire LIWC                          

### Ce ne sont pas tous les commentaires qui contiennent des émojis, on va cependant entraîner les classificateurs sur ces commentaires, car en l'absence de labellisation fiable, qui aurait été faite par des humains, les commentaires avec emojis donnent plus d'information que ceux sans

##### On se ramène aux catégories d'émotions suivantes (à partir des 6 émotions standards + 5 autres que j'avais constitué "empiriquement") :
- 'Bien-être' : 'Joie'
- 'Mal-être' : 'Peur', 'Triste'
- 'Bienveillance' : 'Amour', 'OK'
- 'Malveillance': 'Colère', 'Refus' (dégoût), 'KO'
- 'Neutre': 'Surprise', 'Demande'
###### On peut avoir dans un même texte plusieurs de ces macro-émotions, on n'en retiendra qu'une

 ### Lecture du fichier des commentaires associées aux vidéos d'origine susceptibles de traiter de climatisation (pas juste les 762 vidéos utilisées jusqu'alors)

In [ ]:
# 2.a
chemin_nom_dataframe = "/content/drive/My Drive/Datasets NLP/Climat/yt_com_2019-20_climatisation.csv"

df_comments_climatisation = pd.read_csv(chemin_nom_dataframe, sep=',', encoding='utf-8')
print(len(df_comments_climatisation))
df_comments_climatisation.dtypes

290436


id_chaine            object
id_video             object
id_discussion       float64
id_parent            object
id_commentaire       object
nom_auteur           object
id_chaine_auteur     object
nb_réponses           int64
nb_likes              int64
date_publication     object
commentaire          object
dtype: object

###### Fichier des Emojis

In [ ]:
# 2.b
chemin_nom_fichier = "/content/drive/My Drive/Datasets NLP/Climat/Emojis Emotions.csv"

df_emojis = pd.read_csv(chemin_nom_fichier, sep=';', encoding='utf-8')
df_emojis.head()

,Char,Unicode,Occurrences,Position,Neg,Neut,Pos,Sentiment score,Unicode name,Unicode block,Classe
0,😂,0x1f602,14622,0.805,0.247,0.285,0.468,0.221,FACE WITH TEARS OF JOY,Emoticons,Joie
1,❤,0x2764,8050,0.747,0.044,0.166,0.790,0.746,HEAVY BLACK HEART,Dingbats,Amour
2,♥,0x2665,7144,0.754,0.035,0.272,0.693,0.657,BLACK HEART SUIT,Miscellaneous Symbols,Amour
3,😍,0x1f60d,6359,0.765,0.052,0.219,0.729,0.678,SMILING FACE WITH HEART-SHAPED EYES,Emoticons,Amour
4,😭,0x1f62d,5526,0.803,0.436,0.220,0.343,-0.093,LOUDLY CRYING FACE,Emoticons,Triste


In [ ]:
# 2.c
# création du dictionnaire associant une des 10 émotions du fichier à un émoticon,
d_emoji_emotion = {}
for ix, donnees_emoji in df_emojis.iterrows():
    d_emoji_emotion[donnees_emoji['Char']] = donnees_emoji['Classe']
# rajouts manuel d'emojis:
d_emoji_emotion['🤣'] = 'Joie'
# ou correction
d_emoji_emotion['🙏'] = 'Amour'

# et liste des émojis
l_emojis = [emoji for emoji in d_emoji_emotion]
print(l_emojis)

['😂', '❤', '♥', '😍', '😭', '😘', '😊', '👌', '💕', '👏', '😁', '☺', '♡', '👍', '😩', '🙏', '✌', '😏', '😉', '🙌', '🙈', '💪', '😄', '😒', '💃', '💖', '😃', '😔', '😱', '🎉', '😜', '☯', '🌸', '💜', '💙', '✨', '😳', '💗', '★', '█', '☀', '😡', '😎', '😢', '💋', '😋', '🙊', '😴', '🎶', '💞', '😌', '🔥', '💯', '🔫', '💛', '💁', '💚', '♫', '😞', '😆', '😝', '😪', '�', '😫', '😅', '👊', '💀', '😀', '😚', '😻', '©', '👀', '💘', '🐓', '☕', '👋', '✋', '🎊', '🍕', '❄', '😥', '😕', '💥', '💔', '😤', '😈', '►', '✈', '🔝', '😰', '⚽', '😑', '👑', '😹', '👉', '🍃', '🎁', '😠', '🐧', '☆', '🍀', '🎈', '🎅', '😓', '😣', '😐', '✊', '😨', '😖', '💤', '💓', '👎', '💦', '✔', '😷', '⚡', '🙋', '🎄', '💩', '🎵', '➡', '😛', '😬', '👯', '💎', '🌿', '🎂', '🌟', '🔮', '❗', '👫', '🏆', '✖', '☝', '😙', '⛄', '👅', '♪', '🍂', '💏', '🔪', '🌴', '👈', '🌹', '🙆', '➜', '👻', '💰', '🍻', '🙅', '🌞', '🍁', '⭐', '▪', '🎀', '━', '☷', '🐷', '🙉', '🌺', '💅', '🐶', '🌚', '👽', '🎤', '👭', '🎧', '👆', '🍸', '🍷', '®', '🍉', '😇', '☑', '🏃', '😿', '│', '💣', '🍺', '▶', '😲', '🎸', '🍹', '💫', '📚', '😶', '🌷', '💝', '💨', '🏈', '💍', '☔', '👸', '🇪', '░', '🍩', '👾', '☁', '🌻', '😵',

###### Sélection des seuls commentaires avec des emojis de la liste connue

In [ ]:
# 2.d
# savoir si un texte contient des émojis
def text_with_known_emojis(texte, l_emojis):
    nb_emojis = 0
    for c in str(texte):
        if c in l_emojis:
            nb_emojis += 1
    return nb_emojis

In [ ]:
# 2.e
# ON PEUT PASSER (PREND 4 MINUTES):
# compter les émojis (de la liste) par commentaires
l_nb_emojis = []
for ix, commentaire in tqdm(df_comments_climatisation['commentaire'].iteritems(), total=len(df_comments_climatisation)):
    nb_emojis = text_with_known_emojis(str(commentaire), l_emojis)
    l_nb_emojis.append(nb_emojis)

# avoir une idée de la répartition des commentaires par nombre d'émojis
d_nb_emojis = {}
for nb_emojis in l_nb_emojis:
    if nb_emojis not in d_nb_emojis: d_nb_emojis[nb_emojis] = 0
    d_nb_emojis[nb_emojis] += 1

l_nb_emojis = [(nb_emojis, nb_comments) for nb_emojis, nb_comments in d_nb_emojis.items()]
l_nb_emojis.sort(key=lambda x: x[0])
print(l_nb_emojis)

# et le nombre de commentaires avec au moins 1 emoji
nb_comments_with_emojis = 0
for (nb_emojis, nb_comments) in l_nb_emojis:
    if nb_emojis > 0: nb_comments_with_emojis += nb_comments
print(nb_comments_with_emojis)

  2%|▏         | 5715/290436 [00:03<03:04, 1547.08it/s]


KeyboardInterrupt: ignored

In [ ]:
# 2.f
# sélection des commentaires avec émojis
# PREND 4 MINUTES
df_comments_climatisation['nb_emojis_connus'] =\
df_comments_climatisation['commentaire'].progress_apply(lambda x: text_with_known_emojis(x, l_emojis))

df_comments_climatisation = df_comments_climatisation.loc[df_comments_climatisation['nb_emojis_connus'] > 0]

100%|██████████| 290436/290436 [03:15<00:00, 1484.65it/s]


In [ ]:
# 2.g
# enlever les émojis connus du texte
# fonction qui d'un texte en retire les émojis déjà connus

# pour avoir un classificateur plus robuste sur les plus nombreux commentaires sans emojis

def remove_known_emojis_from_text(text, l_emojis):
    
    text_wo_emojis = ''
    for c in str(text):
        if c in l_emojis: continue
        text_wo_emojis += c
    return text_wo_emojis

In [ ]:
# 2.h
# et création des textes sans emojis
df_comments_climatisation['commentaire_ss_emojis'] =\
df_comments_climatisation['commentaire'].progress_apply(lambda x: remove_known_emojis_from_text(x, l_emojis))

100%|██████████| 31466/31466 [00:15<00:00, 2061.10it/s]


In [ ]:
# 2.i
# retirer les textes (beaucoup) trop courts
df_comments_climatisation['nb_mots_commentaire_ss_emojis'] = \
df_comments_climatisation['commentaire_ss_emojis'].progress_apply(lambda x: len(str(x).split()))

# au moins 2 mots !
df_comments_climatisation =\
df_comments_climatisation.loc[df_comments_climatisation['nb_mots_commentaire_ss_emojis'] > 1]

print(len(df_comments_climatisation))

100%|██████████| 31466/31466 [00:00<00:00, 266540.85it/s]

28221


###### Sauvegarde intermédiaire de la sélection des emojis

In [ ]:
# 2.j
df_comments_emojis = df_comments_climatisation[['id_video', 'id_commentaire', 
                                                'commentaire', 'commentaire_ss_emojis']]

In [ ]:
# 2.k
chemin_nom_fichier = "/content/drive/My Drive/Datasets NLP/Climat/yt_com_2019-20_climatisation_emojis.csv"
df_comments_emojis.to_csv(chemin_nom_fichier, sep=',', encoding='utf-8', 
                          header=True, index=False, quoting=csv.QUOTE_ALL)

files.download(chemin_nom_fichier)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Analyse linguistique et statistique à effectuer avant l'affectation des émotions aux commentaires
###### En conserver les lemmes pour ensuite utilisation du fichier des sentiments et émotions du lexique

In [ ]:
# 3.a
# on peut reprendre là en cas d'interruption
chemin_nom_fichier = "/content/drive/My Drive/Datasets NLP/Climat/yt_com_2019-20_climatisation_emojis.csv"
df_comments_emojis = pd.read_csv(chemin_nom_fichier, sep=',', encoding='utf-8')
df_comments_emojis.dtypes

id_video                 object
id_commentaire           object
commentaire              object
commentaire_ss_emojis    object
dtype: object

#### Approche classique, travail préalable de traitement NLP
1) pas de normalisation - lower-case effectué dans lemmatisation spacy<p>
2) analyse spacy :<p>
3) élimination non-français<p>
4) retraitement du texte : 
- garder lemmes (lemma_)
- enlever stop-words
- enlever certaines classes, mais garder certaines formes (!, ?, ne, pas)
- on liste les entités nommés
- on liste les groupes nominaux (sur pattern, il suffit de garder les formes d'origine jointes par _
=> nouvelles colonnes : langue, l_tokens, l_ent_nommées, l_groupes nominaux, on sauve !!!<p>
<p>5) pour les bigrammes se contenter d'une analyse statistique sur l_tokens<p>

In [ ]:
# 3.b
import spacy
from spacy.matcher import DependencyMatcher

In [ ]:
# 3.c
nlp = spacy.load("fr_core_news_md")

In [ ]:
# 3.d
# ajouter la détection de langue
from spacy.language import Language
from spacy_langdetect import LanguageDetector
def get_lang_detector(nlp, name):
    return LanguageDetector()
Language.factory("language_detector", func=get_lang_detector)

<function __main__.get_lang_detector>

In [ ]:
# 3.e
nlp.add_pipe('language_detector', last=True)

In [ ]:
# 3.f
# retirer quelques stopwords de la liste de départ, un peu longue
stopwords = nlp.Defaults.stop_words
l_stopwords = list(stopwords)
l_stopwords.sort()
print(len(l_stopwords))
#for stopword in l_stopwords: print(stopword)

507


In [ ]:
# 3.g
# lecture d'un fichier édité manuellement qui liste si les stop-words sont retirables ou non (pour cette tâche)
chemin_nom_fichier = "/content/drive/My Drive/Datasets NLP/Climat/Stop-words Spacy.csv"
df_stopwords_spacy = pd.read_csv(chemin_nom_fichier, sep=';', encoding='utf-8')
df_stopwords_spacy_retirable = df_stopwords_spacy.loc[df_stopwords_spacy['Retirable']=='X']
print(len(df_stopwords_spacy_retirable))

l_stopwords_spacy_retirable = df_stopwords_spacy_retirable['Spacy'].tolist()

292


In [ ]:
# 3.h
# retirer effectivement les stop-words en trop
nlp.Defaults.stop_words -= set(l_stopwords_spacy_retirable)
print(len(nlp.Defaults.stop_words))

215


In [ ]:
print(l_stopwords_spacy_retirable)

['abord', 'afin', 'alors', 'anterieur', 'anterieure', 'anterieures', 'antérieur', 'antérieure', 'antérieures', 'apres', 'après', 'attendu', 'aupres', 'autre', 'autrement', 'autres', 'autrui', 'bas', 'basee', 'bat', 'car', 'cent', 'cependant', 'certain', 'certaine', 'certaines', 'certains', 'certes', 'chacun', 'chacune', 'chaque', 'chez', 'cinq', 'cinquantaine', 'cinquante', 'cinquantième', 'cinquième', 'compris', 'concernant', 'debout', 'dedans', 'dehors', 'deja', 'dejà', 'derriere', 'derrière', 'desormais', 'dessous', 'dessus', 'deux', 'deuxième', 'deuxièmement', 'devant', 'devers', 'devra', 'different', 'differente', 'differentes', 'differents', 'différent', 'différente', 'différentes', 'différents', 'dire', 'directe', 'directement', 'dit', 'dite', 'dits', 'divers', 'diverse', 'diverses', 'dix', 'dix-huit', 'dix-neuf', 'dix-sept', 'dixième', 'doit', 'doivent', 'douze', 'douzième', 'durant', 'dès', 'déja', 'déjà', 'désormais', 'effet', 'egalement', 'également', 'eh', 'elle-meme', 'ell

###### On va en plus de l'analyse classique, construire les groupes nominaux

In [ ]:
# 3.i
# éléments pour "matcher" les groupes nominaux à partir de l'analyse linguistique

matcher = DependencyMatcher(nlp.vocab)

pattern_2 = [
  {
    "RIGHT_ID": "nom_tête",       
    "RIGHT_ATTRS": {"POS": "NOUN"}  
  },
  {
    "LEFT_ID": "nom_tête",
    "REL_OP": ">",        
    "RIGHT_ID": "dépendance",
    "RIGHT_ATTRS": {"DEP": {"IN": ["amod", "nmod"]}},    
  }
]

pattern_3 = [
  {
    "RIGHT_ID": "nom_tête",       
    "RIGHT_ATTRS": {"POS": "NOUN"}  
  },
  {
    "LEFT_ID": "nom_tête",
    "REL_OP": ">",        
    "RIGHT_ID": "dépendance_1",
    "RIGHT_ATTRS": {"DEP": {"IN": ["amod", "nmod"]}},    
  },
  {
    "LEFT_ID": "dépendance_1",
    "REL_OP": ">",        
    "RIGHT_ID": "dépendance_2",
    "RIGHT_ATTRS": {"DEP": {"IN": ["amod", "nmod"]}},    
  }
]

matcher.add("GN_2", [pattern_2])
matcher.add("GN_3", [pattern_3])

###### 2 phases, faire et stocker tous les éléments intéressants de l'analyse Spacy, puis ensuite en déduire la liste des tokens de chaque texte (avant regroupement des groupes nominaux et statistiques)

In [ ]:
# 3.j
# analyse linguistique SpaCy suivie immédiatement de la reconnaissance du groupe nominal
def elements_analyse_nlp(texte, nlp, matcher):
    
    # analyse spacy
    # str juste pour être sûr
    doc = nlp(str(texte))

    # langue du texte
    langue = doc._.language['language']
    
    if langue != 'fr':
        return langue, [], [], []
    
    # éléments liés aux tokens
    # on conserve aussi des éléments d'analyse syntaxique de dépendances (tête et relation de dépendance)
    l_elements_tkn = []
    for tkn in doc:
        elements_tkn = (tkn.text, tkn.i, tkn.idx, 
                        tkn.lemma_, tkn.pos_, tkn.head.i, tkn.dep_, tkn.is_stop, tkn.ent_type)
        l_elements_tkn.append(elements_tkn)
        
    # éléments liés aux entités nommées
    l_elements_ent = []
    for ent in doc.ents:
        elements_ent = (ent.text, ent.start_char, ent.end_char, ent.label_)
        l_elements_ent.append(elements_ent)
    
    # groupes nominaux
    l_groupes_nominaux = []
    matches = matcher(doc)
    # un match est une liste de deux termes, un identifiant et la liste des index de tokens
    for x_groupe_nominal in matches:
        l_groupes_nominaux.append(x_groupe_nominal[1])
    
    
    return langue, l_elements_tkn, l_elements_ent, l_groupes_nominaux

 ## Cette analyse prend beaucoup de temps (15 MINUTES), on laisse les cellules qui suivent, et on va reprendre le résultat sauvegardé

In [ ]:
df_comments_emojis[['langue_comment', 'l_tkn_comment', 'l_ent_comment', 'l_gn_comment']]\
= df_comments_emojis.progress_apply(\
lambda x: pd.Series(elements_analyse_nlp(x['commentaire_ss_emojis'], nlp, matcher)), axis=1)

100%|██████████| 28221/28221 [15:34<00:00, 30.21it/s]


In [ ]:
# 3.l
# retirer ce qui n'est pas en français
df_comments_emojis = df_comments_emojis.loc[df_comments_emojis['langue_comment'] == 'fr']
print(len(df_comments_emojis))

24321


In [ ]:
# 3.m
# plus que la liste des indices des tokens des groupes nominaux, ce qui nous intéresse c'est un texte équivalent
# on reconstruit formes lemmatisées pour tous les tokens, sauf pour les noms, où on prend le texte d'origine
def reconstruit_groupe_nominal(groupe_nominal, l_tkn_sous_titres):

    groupe_lemmes = ""

    # le groupe nominal est une liste d'index qu'il convient de trier dans l'ordre linéaire
    groupe_nominal.sort()
  
    # prendre texte d'origine(mis en minuscules) si nom, lemme sinon
    for ix, i_tkn in enumerate(groupe_nominal):
        tkn = l_tkn_sous_titres[i_tkn]
        texte_lemme = tkn[3]
        pos_token = tkn[4]
        texte_token = tkn[0]
        #if pos_token == 'NOUN': texte_terme = texte_token.lower()
        #else: texte_terme = texte_lemme
        texte_terme = texte_lemme
        groupe_lemmes += texte_terme
        # mettre le séparateur '_', sauf si en dernier
        if ix < len(groupe_nominal)-1: 
            groupe_lemmes += '_'

    return groupe_lemmes

In [ ]:
# 3.n
# récupération et comptage des groupes nominaux (et aussi des textes d'entités nommées)

d_groupes_lemmes = {}
d_entites_nommees = {}

for ix_comment, row in tqdm(df_comments_emojis.iterrows(), total=df_comments_emojis.shape[0]):

    l_tkn_comment = row['l_tkn_comment']
    l_ent_comment = row['l_ent_comment']
    l_gn_comment = row['l_gn_comment']

    # groupes nominaux
    for groupe_nominal in l_gn_comment:
        groupe_lemmes = reconstruit_groupe_nominal(groupe_nominal, l_tkn_comment)
        if groupe_lemmes not in d_groupes_lemmes: d_groupes_lemmes[groupe_lemmes] = 0
        d_groupes_lemmes[groupe_lemmes] += 1

    # entités nommées
    for entite_nommee in l_ent_comment:
        entite_nommee_texte = entite_nommee[0]
        # ne garder que si l'entité nommée contient 2 ou plus tokens (tokénisation ultra simplifiée suffira)

        l_tokens_entite_nommee_texte = entite_nommee_texte.split()
        if len(l_tokens_entite_nommee_texte) >= 2:
            entite_nommee_texte = entite_nommee_texte.replace(' ', '_')
        if entite_nommee_texte not in d_entites_nommees: d_entites_nommees[entite_nommee_texte] = 0
        d_entites_nommees[entite_nommee_texte] += 1

100%|██████████| 24321/24321 [00:02<00:00, 8172.31it/s]


In [ ]:
# 3.o
# conversion en listes triées selon la fréquence (en décroissant)
l_groupes_lemmes = [(groupe_lemmes, frequence) for groupe_lemmes, frequence in d_groupes_lemmes.items()]
l_groupes_lemmes.sort(key=lambda x: x[1], reverse=True)
l_entites_nommees = [(entite_nommee_texte, frequence) for entite_nommee_texte, frequence in d_entites_nommees.items()]
l_entites_nommees.sort(key=lambda x: x[1], reverse=True)

In [ ]:
# 3.p
# visualisation des groupes (liste de groupe et score), par tranches de N autour de valeurs de pourcentages 
def montre_scores_groupes(l_groupes, l_vues_autour, n_montres, format = "%d"):
    nb_groupes = len(l_groupes)
    for vues_autour in l_vues_autour:
        ix_a_vue = int(nb_groupes * vues_autour)
        print("tranche des groupes autour de %d :" % (ix_a_vue))
        ix_groupe_a_vue = range(ix_a_vue, min(ix_a_vue + n_montres, nb_groupes))                          
        for ix in ix_groupe_a_vue:
            s_score = format % l_groupes[ix][1]
            print(l_groupes[ix][0], " : ", s_score)
        print("")

In [ ]:
# 3.q
# visualisation de la distribution des groupes selon la fréquence (ou autre)
def montre_distribution_groupes(l_groupes, val_min=0, nom_mesure = 'fréquence'):
    
    l_freqs = [math.log(x[1],10) for x in l_groupes if x[1] > val_min]
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=l_freqs))
    titre = "Distribution des %ss des groupes" % nom_mesure
    xaxis_titre = "log %ss" % nom_mesure
    yaxis_titre = ""
    fig.update_layout(title=titre, xaxis_title=xaxis_titre, yaxis_title=yaxis_titre)
    fig.show()

In [ ]:
# 3.r
# montrer la distribution des groupes de lemmes
montre_distribution_groupes(l_groupes_lemmes, val_min=2)

In [ ]:
# 3.s
# montrer la distribution des entités nommées
montre_distribution_groupes(l_entites_nommees, val_min=2)

In [ ]:
# 3.t
# sélection sur un seuil de fréquence
# on conserve quand même la donnée de fréquence pour les groupes sélectionnés
def select_groupes_sur_frequence(l_groupes, val_min):
    l_sel_groupes = []
    for (groupe, freq) in l_groupes:
        if freq >= val_min:
            l_sel_groupes.append((groupe, freq))
    return l_sel_groupes

In [ ]:
# 3.u
# sélection des groupes nominaux les plus fréquents
val_min = 10
l_sel_groupes_syntaxe = select_groupes_sur_frequence(l_groupes_lemmes, val_min)
print(len(l_sel_groupes_syntaxe))

533


In [ ]:
# 3.v
# et des entités nommées les plus fréquents
val_min = 10
l_sel_entites_nommees = select_groupes_sur_frequence(l_entites_nommees, val_min)
print(len(l_sel_entites_nommees))

313


#### Sélection statistique

###### Préliminaire, pré-construire la chaîne reconstituée des tokens sélectionnés et lemmatisés
- Cette chaîne servira aussi à calculer les émotions associées aux lemmes

In [ ]:
# 4.a
# liste des catégories morpho-syntaxiques
CATEGORIES_VIDES = ['ADP', 'AUX', 'CONJ', 'CCONJ', 'DET', 'PART', 'PRON',
                    'PUNCT', 'SCONJ', 'SYM', 'SPACE']
CATEGORIES_PLEINES = ['ADJ', 'ADV', 'INTJ', 'NOUN', 'NUM', 'PROPN', 'VERB', 'X']

In [ ]:
# 4.b
# sélection des lemmes
# - catégories pleines (mais qui ne sont pas stop-words)
# - catégories vides (lorsqu'elles sont dans la liste des stop-words retirables)
# - rajout de !, ? et aussi de n', ne, pas
def selectionne_lemme(texte, lemme, pos, est_stopword, l_stopwords_retirable):
    
    selection_lemme = ''
    
    # sélection un peu plus complexe,
    # on garde les catégories pleines qui ne sont pas dans la liste des stop-words
    if pos in CATEGORIES_PLEINES and est_stopword == False:
        #if pos == 'NOUN': selection_lemme = texte.lower()
        #else: selection_lemme = lemme
        selection_lemme = lemme
    # et les catégories vides mais dans la liste des "anciens" stopwords
    if pos not in CATEGORIES_PLEINES and texte.lower() in l_stopwords_retirable:
        selection_lemme = lemme
    # et rajouter "n'", "ne", "pas", ! et ? pour analyse d'émotions pourrait aider
    if texte in ["n'", "ne", "pas", '!', '?']:
        selection_lemme = texte
    
    return selection_lemme

In [ ]:
# 4.c
# transformer la liste de tokens résultat de NLP en une liste des lemmes (ou textes pour les noms) retenus
def cree_l_lemmes_sur_resultat_nlp(l_tokens, l_stopwords_retirable):
    l_items = []
    for token in l_tokens:
        lemme = selectionne_lemme(token[0], token[3], token[4], token[7], l_stopwords_retirable)
        if lemme != '': l_items.append(lemme)
    return l_items

In [ ]:
# 4.d
# création effective des listes de lemmes retenus
df_comments_emojis['l_lemmes']\
= df_comments_emojis['l_tkn_comment'].\
progress_apply(lambda x: cree_l_lemmes_sur_resultat_nlp(x, l_stopwords_spacy_retirable))

100%|██████████| 24321/24321 [00:03<00:00, 7660.52it/s]


In [ ]:
# 4.e
# pour utilisation sous gensim
l_textes_l_tokens = df_comments_emojis['l_lemmes'].values.tolist()

###### coeur de la partie statistique

In [ ]:
# 4.f
from gensim.models.phrases import Phrases, Phraser

In [ ]:
# 4.g

# VERSIONS DIFFERENTES DE GENSIM (COLAB EST UNE VERSION ANTERIEURE A LA PLUS RECENTE)

# créée le modèle et calcule la liste des groupes pour une liste de textes (sous forme de listes de tokens, par exemple lemmatisés)

# scoring ({'default', 'npmi', function}, optional)
# min_count (float, optional) – Ignore all words and bigrams with total collected count lower than this value. 
# threshold (float, optional) – Represent a score threshold for forming the phrases (higher means fewer phrases). 
# A phrase of words a followed by b is accepted if the score of the phrase is greater than threshold. 
# Heavily depends on concrete scoring-function, 'default': > 0, 'npmi': between 0 and 1. 

def cree_modele_groupes_gensim(l_textes_l_tokens, methode='default', nb_min=5, seuil=None):
    
    # obtenir le modèle qui permet de remplacer les couples pertinents par leur concatenation
    # (comme on combine avec les éléments issus de l'analyse syntaxique, ne sera pas utilisé tel quel)
    if seuil is None:
        if methode == 'default': seuil_methode = 10
        if methode == 'npmi': seuil_methode = 0.5
    else:
        seuil_methode = seuil
    # selon version de gensim...
    #modele_groupes = Phrases(l_textes_l_tokens, scoring=methode, min_count=nb_min, threshold=seuil_methode,
    #                         delimiter='_')
    modele_groupes = Phrases(l_textes_l_tokens, scoring=methode, min_count=nb_min, threshold=seuil_methode,
                             delimiter=b'_')

    # obtenir les scores pour analyse des groupes de mots retenus
    # selon version de gensim...
    # x_groupes = modele_groupes.export_phrases()
    # l_groupes = [(groupe, score) for groupe, score in x_groupes.items()]
    x_groupes = modele_groupes.export_phrases(l_textes_l_tokens)
    l_groupes = [(groupe.decode("utf-8"), score) for groupe, score in x_groupes]
    # mettre aussi les groupes sous la forme lemme1_lemme2 au lieu de lemme1 lemme2
    l_groupes = [('_'.join(groupe.split()), score) for (groupe, score) in l_groupes] 
    # cette première liste est une liste des occurrences, la transformer en une liste de termes avec score et nb d'occurrences
    d_groupes = {}
    for (groupe, score) in l_groupes:
        if groupe not in d_groupes:
            d_groupes[groupe] = (score, 1)
        else:
            d_groupes[groupe] = (score, d_groupes[groupe][1] + 1)
    l_groupes = list(d_groupes.items())
    # tri par score, et non par fréquence
    l_groupes.sort(key=lambda x: x[1][0], reverse=True)

    # le modèle ne servira pas plus dans cette fonction, renvoyé pour utilisation dans d'autres contextes  
    return modele_groupes, l_groupes

In [ ]:
# 4.h
# création effective des groupes statistiques
methode_gensim='default'
nb_min_gensim=5
seuil_gensim=3
modele_groupes_stat, l_groupes_stat = cree_modele_groupes_gensim(l_textes_l_tokens, 
                                                    methode=methode_gensim, 
                                                    nb_min=nb_min_gensim,
                                                    seuil=seuil_gensim)

In [ ]:
# 4.i
# pour visualisation, séparer en deux listes, des fréquences et des scores
l_groupes_stat_scores = [(groupe, score) for (groupe, (score, frequence)) in l_groupes_stat]
l_groupes_stat_freqs = [(groupe, frequence) for (groupe, (score, frequence)) in l_groupes_stat]
l_groupes_stat_freqs.sort(key=lambda x: x[1], reverse=True)

In [ ]:
# 4.j
# présentation de la distribution des bigrammes statistiquement pertinents
montre_distribution_groupes(l_groupes_stat_scores, val_min=20, nom_mesure='score')

In [ ]:
# 4.k
# il peut être plus intéressant de montrer la relation entre score et fréquence, 
# pour voir quelle meilleure combinaison pour la sélection
def montre_distribution_score_frequence(l_groupes_stat):

    # en x, liste des fréquences (logarithmes de )
    l_freqs = [math.log(frequence, 10) for (groupe, (score, frequence)) in l_groupes_stat]
    # en y, liste des scores
    l_scores = [math.log(score, 10) for (groupe, (score, frequence)) in l_groupes_stat]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=l_freqs, y=l_scores, mode='markers'))
    titre = "Distribution fréquences / scores des groupes statistiques"
    xaxis_titre = "log fréquence"
    yaxis_titre = "log score"
    fig.update_layout(title=titre, xaxis_title=xaxis_titre, yaxis_title=yaxis_titre)
    fig.show()

In [ ]:
# 4.l
# présentation de la distribution des bigrammes par fréquence
montre_distribution_score_frequence(l_groupes_stat)

In [ ]:
# 4.m
# combinaison score / fréquence
def montre_distribution_combinaison_score_frequence(l_groupes_stat, coeff_frequence=1.0):
    l_combis = [math.log(score, 10) + coeff_frequence * math.log(frequence, 10)\
                for (groupe, (score, frequence)) in l_groupes_stat]
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=l_combis))
    titre = "Distribution combinaison scores.fréquences des groupes statistiques"
    xaxis_titre = "log score + %.1f * log fréquence" % coeff_frequence
    yaxis_titre = ""
    fig.update_layout(title=titre, xaxis_title=xaxis_titre, yaxis_title=yaxis_titre)
    fig.show()

In [ ]:
# 4.n
# combinaison score / fréquence
montre_distribution_combinaison_score_frequence(l_groupes_stat, coeff_frequence=1.0)

In [ ]:
# 4.o
# sélection sur combinaison du score et de la fréquence
def select_groupes_sur_combi(l_groupes_stat, val_min, coeff_frequence=1.0):
    l_sel_groupes = []
    for (groupe, (score, frequence)) in l_groupes_stat:
        if math.log(score, 10) + coeff_frequence * math.log(frequence, 10) > val_min:
            # on garde la fréquence, pour être en phase avec les données groupes nominaux
            l_sel_groupes.append((groupe, frequence))
    return l_sel_groupes

In [ ]:
# 4.p
# sélection des groupes statistiques
# même ordre de grandeur en nombre est OK
l_sel_groupes_stat = select_groupes_sur_combi(l_groupes_stat, 3.8, coeff_frequence=1.0)
print(len(l_sel_groupes_stat))

457


In [ ]:
# 4.q
# union des trois groupes
d_sel_groupes = {}
for (groupe, freq) in l_sel_groupes_stat:
    if groupe not in d_sel_groupes:
        d_sel_groupes[groupe] = freq
for (groupe, freq) in l_sel_groupes_syntaxe:
      if groupe not in d_sel_groupes:
        d_sel_groupes[groupe] = freq
for (groupe, freq) in l_sel_entites_nommees:
      if groupe not in d_sel_groupes:
        d_sel_groupes[groupe] = freq
l_sel_groupes = list(d_sel_groupes.items())
l_sel_groupes.sort(key=lambda x: x[1], reverse=True) 

print("nb de groupes union :", len(l_sel_groupes))

nb de groupes union : 1218


In [ ]:
# 4.r
# des groupes sont en intersection, d'autres spécifiques...
s_sel_groupes_stat = set(l_sel_groupes_stat)
s_sel_groupes_syntaxe = set(l_sel_groupes_syntaxe)
s_sel_entites_nommees = set(l_sel_entites_nommees)

s_stat_et_syntaxe = s_sel_groupes_stat.intersection(l_sel_groupes_syntaxe)
print("nb de groupes communs stat & syntaxe:", len(s_stat_et_syntaxe))
s_stat_non_syntaxe = s_sel_groupes_stat.difference(s_sel_groupes_syntaxe)
print("nb de groupes syntaxe pas dans stat :", len(s_stat_non_syntaxe))
s_syntaxe_non_stat = s_sel_groupes_syntaxe.difference(s_sel_groupes_stat)
print("nb de groupes stat pas dans syntaxe :", len(s_syntaxe_non_stat))

s_stat_et_entite = s_sel_groupes_stat.intersection(l_sel_entites_nommees)
print("nb de groupes communs stat & entités nommées:", len(s_stat_et_entite))

nb de groupes communs stat & syntaxe: 39
nb de groupes syntaxe pas dans stat : 418
nb de groupes stat pas dans syntaxe : 494
nb de groupes communs stat & entités nommées: 0


In [ ]:
# 4.s
# réécriture de la liste des tokens en intégrant les bigrammes de toutes origines, linguistiques, statistiques et entités nommées
l_textes_l_tokens_2 = []
for l,l_tokens in enumerate(l_textes_l_tokens):
    l_tokens_2 = []
    ix = 0
    while ix < len(l_tokens):
        # tant pis si on ne capture pas les bigrammes de fin
        if ix <= len(l_tokens) - 2:
            groupe = '_'.join([l_tokens[ix], l_tokens[ix+1]])
            if groupe in d_sel_groupes:
                l_tokens_2.append(str(groupe))
                ix += 2
            else:
                # str: éviter cas de certains nombres
                l_tokens_2.append(str(l_tokens[ix]))
                ix += 1
        else:
            l_tokens_2.append(str(l_tokens[ix]))
            ix += 1
    l_textes_l_tokens_2.append(l_tokens_2)   

In [ ]:
# 4.t
# intégration effective de cette liste
l_commentaires_avec_bigrammes = [' '.join(l_tokens) for l_tokens in l_textes_l_tokens_2]
df_comments_emojis['commentaire_avec_bigrammes'] = l_commentaires_avec_bigrammes

###### sauvegarde pour éviter de reconstruire l'analyse NLP

In [ ]:
# 4.g
nom_chemin_fichier =  "/content/drive/My Drive/Datasets NLP/Climat/yt_com_2019-20_climatisation_emojis_bigrammes.csv"
df_comments_emojis.to_csv(nom_chemin_fichier, sep=',', encoding='utf-8', 
                          header=True, index=False, quoting=csv.QUOTE_ALL)
files.download(nom_chemin_fichier)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Détermination des (macro-)émotions des commentaires avec emojis

In [ ]:
# 5.a
# si nécessaire, rechargement du fichier constitué précédemment
nom_chemin_fichier = "/content/drive/My Drive/Datasets NLP/Climat/yt_com_2019-20_climatisation_emojis_bigrammes.csv"
df_comments_emojis = pd.read_csv(nom_chemin_fichier, sep=',', encoding='utf-8')

###### Lecture du fichier des sentiments et émotions associés au lexique
- On crée un dictionnaire associant à chaque lemme un tuple (polarity;joy;fear;sadness;anger;surprise;disgust)

In [ ]:
# 5.b
d_lemme_emotion = {}
nom_chemin_fichier = "/content/drive/My Drive/Datasets NLP/Climat/FEEL.csv"
f_emotions = open(nom_chemin_fichier, 'r', encoding='utf-8')
for no_ligne, ligne in enumerate(f_emotions):
    if no_ligne == 0: continue
    l_items = ligne.split(';')
    lemme = l_items[1]
    # en fait, ce ne sont pas que des lemmes, ici on se contente d'approximations, pas de travail plus fin
    # sur les bigrammes (qu'on aurait plus faire)
    polarite = '0'
    if l_items[2] == 'positive': polarite = '+'
    if l_items[2] == 'negative': polarite = '-'
    # pour le reste, mettre directement dans le même ordre Joie, Peur, Triste, Colère, Surprise, Refus (dégoût)
    if lemme in d_lemme_emotion: print(lemme, "en double")
    d_lemme_emotion[lemme] = (polarite, 
                              int(l_items[3]), int(l_items[4]), int(l_items[5]),
                              int(l_items[6]), int(l_items[7]), int(l_items[8]))
    
f_emotions.close()

In [ ]:
print(d_lemme_emotion['bon'])

('+', 0, 0, 0, 0, 1, 0)


#### Détermination des (macro-)émotions avec à la fois les emojis et les lemmes
- émojis : dans 'commentaire'
- lemmes : dans l_textes_l_tokens

In [ ]:
# 5.c
# auparavant, correspondance entre émotion "de base", et macro-émotions
d_macro_emotion = {'Joie': 'Bien-être',
                   'Peur': 'Mal-être',
                   'Triste': 'Mal-être',
                   'Amour': 'Bienveillance',
                   'OK': 'Bienveillance',
                   'KO': 'Malveillance',
                   'Colère': 'Malveillance',
                   'Refus': 'Malveillance',
                   'Surprise': 'Neutre',
                   'Demande': 'Neutre',
                   'Neutre': 'Neutre'}

In [ ]:
# 5.d
# fonction de détection de l'émotion par comptages
def init_emotion_texte(commentaire, l_lemmes,
                       d_emoji_emotion, d_lemme_emotion, d_macro_emotion,
                       locace = False):
    # comptages à 0
    d_macro_comptages = {'Bien-être': 0, 'Mal-être': 0, 
                         'Bienveillance': 0, 'Malveillance': 0,
                         'Neutre': 0}
    
    # emojis (qui sont des caractères) lus dans commentaires
    for c in commentaire:
        if c in d_emoji_emotion:
            macro_emotion = d_macro_emotion[d_emoji_emotion[c]]
            d_macro_comptages[macro_emotion] += 1
            if locace == True: print(c, macro_emotion)
            
    # lemmes
    for lemme in l_lemmes:
        if lemme in d_lemme_emotion:
            if d_lemme_emotion[lemme][1] == 1: 
                d_macro_comptages[d_macro_emotion['Joie']] += 1
                if locace == True: print(lemme, 'Joie')
            if d_lemme_emotion[lemme][2] == 1: 
                d_macro_comptages[d_macro_emotion['Peur']] += 1
                if locace == True: print(lemme, 'Peur')
            if d_lemme_emotion[lemme][3] == 1: 
                d_macro_comptages[d_macro_emotion['Triste']] += 1
                if locace == True: print(lemme, 'Triste')
            if d_lemme_emotion[lemme][4] == 1: 
                d_macro_comptages[d_macro_emotion['Colère']] += 1
                if locace == True: print(lemme, 'Colère')
            if d_lemme_emotion[lemme][5] == 1: 
                d_macro_comptages[d_macro_emotion['Surprise']] += 1
                if locace == True: print(lemme, 'Surprise')
            if d_lemme_emotion[lemme][6] == 1: 
                d_macro_comptages[d_macro_emotion['Refus']] += 1
                if locace == True: print(lemme, 'Refus')
                
    # quelle macro-émotion renvoyer ????
    l_macro_comptages = [(macro_emotion, comptage) for macro_emotion, comptage in d_macro_comptages.items()]
    l_macro_comptages.sort(key=lambda x: x[1], reverse=True)
    
    if locace == True: print(l_macro_comptages)
    
    macro_comptage_max_1 = l_macro_comptages[0][1]
    macro_comptage_max_2 = l_macro_comptages[1][1]
    # si rien, renvoyer 'Neutre'
    if macro_comptage_max_1 == 0: return 'Neutre'
    
    # si le deuxième est à 0, renvoyer le premier type trouvé
    if macro_comptage_max_2 == 0: return l_macro_comptages[0][0]
    
    # plus généralement, si gros écart entre le premier et le deuxième, renvoyer le premier
    if macro_comptage_max_2 < 0.7 * macro_comptage_max_1: return l_macro_comptages[0][0]
    
    # si les deux premiers se tiennent, si l'un des deux est 'Neutre', renvoyer l'autre
    if l_macro_comptages[0][0] == 'Neutre': return l_macro_comptages[1][0]
    if l_macro_comptages[1][0] == 'Neutre': return l_macro_comptages[0][0]
    
    # cas résiduel, renvoyer une combinaison des deux premiers (faire attention à ne pas avoir A-B et B-A),
    # mais seulement si combinaison de bien et de mal, sinon renvoyer le 1er
    if (l_macro_comptages[0][0][0:4]=='Bien' and l_macro_comptages[1][0][0:4]=='Bien')\
    or (l_macro_comptages[0][0][0:3]=='Mal' and l_macro_comptages[1][0][0:3]=='Mal'):
        return l_macro_comptages[0][0]
    
    # deux combinaisons résiduelles s'appellent 'Mal-être-Bienveillance' et 'Bien-être-Malveillance'
    # on laisse tomber dans un premier temps
    # privilégier le négatif est plus réaliste
    if l_macro_comptages[0][0][0:3]=='Mal': return l_macro_comptages[0][0]
    if l_macro_comptages[1][0][0:3]=='Mal': return l_macro_comptages[1][0]
    
    # au cas où resterait un cas résiduel
    return l_macro_comptages[0][0]

In [ ]:
# 5.e
# teste de la pertinence de la (pré-)classification des émotions
for ix in range(1000, 1030):
    commentaire = df_comments_emojis.iloc[ix]['commentaire']
    l_lemmes = df_comments_emojis.iloc[ix]['l_lemmes']
    
    print(commentaire)
    print(l_lemmes)
    macro_emotion = init_emotion_texte(commentaire, l_lemmes,
                                       d_emoji_emotion, d_lemme_emotion, d_macro_emotion, locace=True)
    print(macro_emotion)
    print('****************')

Cc Jean , elle vraiment  inspirante cette vidéo. J'ai cette même vision. Quand je rends visite á mes grands parents au village j'ai du mal á répartir, waoh quelle sérénité ! quelle connexion ! 🙏🏻🙏🏻🙏🏻 Excellente  journée !
['Cc', 'Jean', 'vraiment', 'inspirer', 'vidéo', 'vision', 'rendre', 'visite', 'grand', 'parent', 'village', 'mal', 'répartir', 'waoh', 'sérénité', '!', 'connexion', '!', '🏻🏻🏻', 'excellent', 'journée', '!']
🙏 Bienveillance
🙏 Bienveillance
🙏 Bienveillance
[('Bienveillance', 3), ('Bien-être', 0), ('Mal-être', 0), ('Malveillance', 0), ('Neutre', 0)]
Bienveillance
****************
exactement Jean et c'est d'ailleurs pour ça que j'ai pas continué ma formation de paysagiste ! métier que je voulais faire car j'aime être dehors et être dans la nature mais après reflexion a quoi ça serre a de toujours tondre tailler et couper les choses il y a plus important dans la vie 😂 après pour les jardins privé ou chez des propiétaires c'est compréhensible
['exactement', 'Jean', 'ailleurs

#### Conclusion, ce genre d'approche n'est pas suffisante en tant que tel, mais peut servir à initialiser le travail d'étiquetage

##### Néanmoins, on va s'en contenter, faute de temps pour un travail d'étiquetage

In [ ]:
# 5.f
# pour utilisation dans le data-frame
def met_emotion_commentaire(ligne, d_emoji_emotion, d_lemme_emotion, d_macro_emotion):
    
    commentaire = ligne['commentaire']
    l_lemmes = ligne['l_lemmes']
    macro_emotion = init_emotion_texte(commentaire, l_lemmes,
                                       d_emoji_emotion, d_lemme_emotion, d_macro_emotion,
                                       locace = False)
    return macro_emotion

In [ ]:
# 5.g
# attribution des (macro-)émotions
df_comments_emojis['émotion'] =\
df_comments_emojis.progress_apply(lambda ligne: met_emotion_commentaire(ligne, 
                                  d_emoji_emotion, d_lemme_emotion, d_macro_emotion), axis=1)

100%|██████████| 24321/24321 [00:01<00:00, 21756.36it/s]


In [ ]:
# 5.h
# répartition des émotions est ici relativement équilibrée entre "mal" et "bien"
df_comments_emojis['émotion'].value_counts()

Bien-être        12029
Bienveillance     7723
Mal-être          1899
Malveillance      1457
Neutre            1213
Name: émotion, dtype: int64

In [ ]:
# 5.i
# sauvegarde avant classification
nom_chemin_fichier = "/content/drive/My Drive/Datasets NLP/yt_com_2019-20_climatisation_emojis_bigrammes.csv"
df_comments_emojis.to_csv(nom_chemin_fichier, sep=',', encoding='utf-8', 
                          header=True, index=False, quoting=csv.QUOTE_ALL)

## Classification à proprement parler

In [ ]:
# 6.a
# (re-)chargement du dataset avec les émotions
nom_chemin_fichier =  "/content/drive/My Drive/Datasets NLP/yt_com_2019-20_climatisation_emojis_bigrammes.csv"
df_comments_emojis = pd.read_csv(nom_chemin_fichier, sep=',', encoding='utf-8')
print(len(df_comments_emojis))
df_comments_emojis.dtypes

24321


id_video                      object
id_commentaire                object
commentaire                   object
commentaire_ss_emojis         object
langue_comment                object
l_tkn_comment                 object
l_ent_comment                 object
l_gn_comment                  object
l_lemmes                      object
commentaire_avec_bigrammes    object
émotion                       object
dtype: object

###### Statistiques de base

In [ ]:
# 6.b
# liste des commentaires avec bigrammes (et sans émojis de la liste principale)
l_textes = df_comments_emojis['commentaire_avec_bigrammes'].tolist()
l_textes = [str(texte) for texte in l_textes]

In [ ]:
# 6.c
# comme la tokénisation a déjà eu lieu, il suffit de spliter
l_lng_textes = [len(texte.split()) for texte in l_textes]

lng_mediane = np.median(np.array(l_lng_textes))
print(lng_mediane)
lng_max = np.max(np.array(l_lng_textes))
print(lng_max)

9.0
785


In [ ]:
# 6.d
# comptage de tous les termes employés, avec leurs fréquences
d_termes = {}
for texte in l_textes:
    l_tokens = texte.split()
    for token in l_tokens:
        if token not in d_termes: d_termes[token] = 0
        d_termes[token] += 1
print(len(d_termes))

l_termes = [(terme, freq) for terme, freq in d_termes.items()]
l_termes.sort(key=lambda x: x[1], reverse=True)

print(l_termes[17000])
print(l_termes[18000])

39920
('brèche', 2)
('marié^^', 1)


In [ ]:
# 6.e
# liste des émotions, triées par ordre alphabétique (l'ordre retenu par les algo. de classification)
l_labels = df_comments_emojis['émotion'].unique()
l_labels.sort()
print(l_labels)

['Bien-être' 'Bienveillance' 'Mal-être' 'Malveillance' 'Neutre']


### Préparation des données, entraînement, validation, test

In [ ]:
# 6.f
# ici, plutôt que faire une tripartition :
# entraine : pour entrainer le modèle (avec un jeu d'hyper-paramètres donné)
# valide : pour valider et trouver le meilleur jeu d'hyper-paramètres du modèle
# teste : pour mesurer la performance du modèle (et le comparer à d'autres)
# df_entraine, df_valide = train_test_split(df_comments_emojis, test_size=0.2)
# df_valide, df_teste = train_test_split(df_valide, test_size=0.2)
# print("entrainement :", len(df_entraine), "validation :", len(df_valide), "test :", len(df_teste))

# on va plutôt, faire une bipartition :
# entraine : pour entrainer le modèle + les hyper-paramètres
# teste : pour mesurer la performance du modèle + hyper-paramètres (et le comparer à d'autres combinaisons)
# à fortiori, pas de validation croisée

In [ ]:
# 6.g
# Train test split
from sklearn.model_selection import train_test_split

In [ ]:
# 6.h
# 80 % pour l'entrainement, 20 % pour le tests
df_entraine, df_teste = train_test_split(df_comments_emojis, test_size=0.2)

In [ ]:
# 6.i
# revoir la répartition des classes, pour procéder à éventuel rééquilibrage (down et up-sampling)
df_entraine['émotion'].value_counts()

Mal-être         7685
Bien-être        6273
Bienveillance    2503
Malveillance     2387
Neutre            608
Name: émotion, dtype: int64

In [ ]:
# 6.j
df_teste['émotion'].value_counts()

Mal-être         1937
Bien-être        1498
Bienveillance     659
Malveillance      609
Neutre            162
Name: émotion, dtype: int64

In [ ]:
# on va faire un rééquilibrage naïf du jeu d'entraînement (et validation si existe, mais pas teste)
# au départ, les classes sont ainsi réparties :
# Mal-être 7693, Bien-être 6209
# Bienveillance 2514, Malveillance 2436
# Neutre 604

# On va undersampler Mal-être => 3600, Bien-être => 3200
# Garder Bienveillance, Malveillance 
# Oversampler Neutre => 1800

In [ ]:
# 6.k
df_entraine_mal_etre = df_entraine.loc[df_entraine['émotion'] == 'Mal-être']
df_entraine_mal_etre = df_entraine_mal_etre.sample(3600)
df_entraine_bien_etre = df_entraine.loc[df_entraine['émotion'] == 'Bien-être']
df_entraine_bien_etre = df_entraine_bien_etre.sample(3200)

df_entraine_bienveillance = df_entraine.loc[df_entraine['émotion'] == 'Bienveillance']
df_entraine_malveillance = df_entraine.loc[df_entraine['émotion'] == 'Malveillance']

df_entraine_neutre = df_entraine.loc[df_entraine['émotion'] == 'Neutre']
df_entraine_neutre = df_entraine_neutre.sample(1800, replace=True)

In [ ]:
# 6.l
# remise ensemble, concaténation des différents échantillonage
df_entraine_equilibre = pd.concat([df_entraine_mal_etre, df_entraine_bien_etre, 
                                   df_entraine_bienveillance, df_entraine_malveillance, 
                                   df_entraine_neutre], axis=0)
print(len(df_entraine_equilibre))
df_entraine_equilibre['émotion'].value_counts()

13490


Mal-être         3600
Bien-être        3200
Bienveillance    2503
Malveillance     2387
Neutre           1800
Name: émotion, dtype: int64

In [ ]:
# 6.m
# reshuffling des lignes
df_entraine_equilibre = df_entraine_equilibre.sample(frac=1).reset_index(drop=True)

### Classifications classiques

##### Vectorisation tf-idf

In [ ]:
# 7.a
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [ ]:
# 7.b
# tokeniseur rudimentaire, car tout a été préparé en amont, sert pour remplacer le tokeniseur par défaut de scikit-learn
def simple_split(s):
    return s.split()

In [ ]:
# 7.c
# La phase préliminaire est une phase de vectorisation
# TF-IDF est dans un montage classique un peu plus performant que le simple comptage
# adapter une fois pour toutes le modèle à l'ensemble des données
# on l'utilisera ensuite pour les différents splits, entrainement et test

# paramétrage
# max_df: pour enlever les termes qui sont trop fréquents, présents dans trop de documents, défaut skearn 1.0 (100 %, présents partout ne suffit alors pas pour être éliminé 
# min_df: pour enlever les termes qui sont trop rares, présents dans pas assez de documents, défaut sklearn 1 (nb docs < 1, présents dans 1 seul doc suffit pour être conservé),
# à modifier pour larges corpus (le défaut gensim est à 5)
# max_features: nb total de termes retenus, selon la fréquence, par défaut (None) on garde tout, une bonne fourchette est entre 10000 et 15000 termes

max_dfreq = 1.0
min_dfreq = 5
nb_termes = 18000

tfidf = TfidfVectorizer(max_df=max_dfreq, min_df=min_dfreq, max_features=nb_termes, 
                        lowercase=False, tokenizer=simple_split)
tfidf.fit(l_textes)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


TfidfVectorizer(lowercase=False, max_features=18000, min_df=5,
                tokenizer=<function simple_split at 0x7f74fcb368c0>)

In [ ]:
# 7.d
# créer les entrées vectorisées par tfidf pour les splits entraînement et test
# ainsi que les entrées labels
# on va prendre notations classiques

# textes de départ
X_textes_entraine = df_entraine_equilibre['commentaire_avec_bigrammes'].tolist()
X_textes_entraine = [str(texte) for texte in X_textes_entraine]
X_textes_teste = df_teste['commentaire_avec_bigrammes'].tolist()
X_textes_teste = [str(texte) for texte in X_textes_teste]

# transformés par tfidf
X_entraine = tfidf.transform(X_textes_entraine)
X_teste = tfidf.transform(X_textes_teste)

# labels
y_entraine = df_entraine_equilibre['émotion'].tolist()
y_teste = df_teste['émotion'].tolist()

###### Bayes
- Multinomial Naive Bayes est le standard pour la catégorie des algorithmes bayésiens naïfs (en classification de texts)
- Complement Naive Bayes est une autre possibilité, intéressante pour les catégories déséquilibrées<p><p>
Pas réellement d'hyper-paramètres à faire varier

In [ ]:
# 7.e
# définition du classificateur, Bayes Multinomial, et entraînement (fitting) du modèle
from sklearn.naive_bayes import MultinomialNB
NB_classifier = MultinomialNB()

NB_classifier.fit(X_entraine,y_entraine)

MultinomialNB()

In [ ]:
# 7.f
# application (prédiction des labels) au jeu de test
y_teste_predit = NB_classifier.predict(X_teste)

###### Pour évaluer les résultats de cette méthode (et de toutes les autres), on va calculer les métriques classiques de classification, et afficher la matrice de confusion

In [ ]:
# 7.g
from sklearn import metrics

In [ ]:
# 7.h
# présentation de la matrice de confusion
# code largement inspiré d'une réponse sous stack overflow

def presente_matrice_confusion(matrice_confusion, l_labels, 
                               texte_methode_et_parametres, 
                               macro_precision, macro_rappel, macro_f1):
    
    l_x = l_labels
    l_y = l_labels
    
    mx_z = []
    max_val = 0
    for x in matrice_confusion:
        v_x = []
        for y in x:
            v_x.append(y)
            if y > max_val: max_val = y
        mx_z.append(v_x)
    zmid = max_val / 2
    
    mx_z_texte = []
    for v_x in mx_z:
        v_x_texte = []
        for y in v_x:
            v_x_texte.append(str(y))
        mx_z_texte.append(v_x_texte)

    # create_annotated_heatmap est buggé si on passe x, y...
    #fig = ff.create_annotated_heatmap(z=matrice_confusion, x=l_x, y=l_y, 
    #                                  colorscale='Viridis')
    
    fig = go.Figure()
    fig.add_trace(go.Heatmap(z=matrice_confusion, x=l_x, y=l_y, colorscale='Greys'))
    for n, row in enumerate(mx_z):
        for m, val in enumerate(row):
            font_color = 'black' if val < zmid else 'white'
            fig.add_annotation(
                text=mx_z_texte[n][m],
                x=m,
                y=n,
                xref="x1",
                yref="y1",
                font=dict(color=font_color),
                showarrow=False,
                )
    
    # titre
    titre = "Matrice de confusion - %s" % texte_methode_et_parametres + "<br><sup>" + \
    "macro - précision : %.2f, rappel : %.2f, F1 : %.2f" % (macro_precision, macro_rappel, macro_f1) + "</sup>"
    fig.update_layout(title_text=titre)
    # légendes x et y en tant qu'annotation
    fig.add_annotation(dict(font=dict(color="black",size=14),
                            x=0.5, y=-0.15, showarrow=False,
                            text="Valeur prédite",
                            xref="paper", yref="paper"))
    fig.add_annotation(dict(font=dict(color="black",size=14),
                            x=-0.10, y=0.5, showarrow=False,
                            text="Valeur réelle", 
                            textangle=-90, xref="paper", yref="paper"))
    # avec marge pour l'axe des y
    fig.update_layout(margin=dict(t=50, l=200))
    # légende des couleurs
    fig['data'][0]['showscale'] = True
    
    fig.show()

In [ ]:
# 7.l
# indicateurs classiques (exactitude, précision, rappel, score f-1)
NB_exactitude = metrics.accuracy_score(y_teste, y_teste_predit)
NB_macro_precision = metrics.precision_score(y_teste, y_teste_predit, average='macro')
NB_macro_rappel = metrics.recall_score(y_teste, y_teste_predit, average='macro')
NB_macro_f1 = metrics.f1_score(y_teste, y_teste_predit, average='macro')

print(metrics.classification_report(y_teste, y_teste_predit, target_names=l_labels))

               precision    recall  f1-score   support

    Bien-être       0.58      0.42      0.49      1590
Bienveillance       0.42      0.42      0.42       644
     Mal-être       0.58      0.83      0.68      1879
 Malveillance       0.52      0.24      0.33       602
       Neutre       0.23      0.13      0.16       150

     accuracy                           0.55      4865
    macro avg       0.46      0.41      0.42      4865
 weighted avg       0.54      0.55      0.52      4865



In [ ]:
# 7.m
# et matrice de confusion
NB_matrice_confusion = metrics.confusion_matrix(y_teste, y_teste_predit)
presente_matrice_confusion(NB_matrice_confusion, l_labels, 
                           "Multinomial Bayes Naïf", 
                            NB_macro_precision, NB_macro_rappel, NB_macro_f1)

In [ ]:
# 7.n
# rassembler le tout en une unique fonction
def calcule_presente_evaluation(y_teste, y_teste_predit,
                                l_labels, texte_methode_et_parametres):
    
    exactitude = metrics.accuracy_score(y_teste, y_teste_predit)
    macro_precision = metrics.precision_score(y_teste, y_teste_predit, average='macro')
    macro_rappel = metrics.recall_score(y_teste, y_teste_predit, average='macro')
    macro_f1 = metrics.f1_score(y_teste, y_teste_predit, average='macro')

    print(metrics.classification_report(y_teste, y_teste_predit, target_names=l_labels))
    
    matrice_confusion = metrics.confusion_matrix(y_teste, y_teste_predit)
    presente_matrice_confusion(matrice_confusion, l_labels, 
                               texte_methode_et_parametres, 
                               macro_precision, macro_rappel, macro_f1)
    
    return exactitude, macro_precision, macro_rappel, macro_f1

In [ ]:
# 7.o
NB_exactitude, NB_macro_precision, NB_macro_rappel, NB_macro_f1 =\
calcule_presente_evaluation(y_teste, y_teste_predit,
                            l_labels, "Multinomial Bayes Naïf")

               precision    recall  f1-score   support

    Bien-être       0.58      0.42      0.49      1590
Bienveillance       0.42      0.42      0.42       644
     Mal-être       0.58      0.83      0.68      1879
 Malveillance       0.52      0.24      0.33       602
       Neutre       0.23      0.13      0.16       150

     accuracy                           0.55      4865
    macro avg       0.46      0.41      0.42      4865
 weighted avg       0.54      0.55      0.52      4865



In [ ]:
# 7.p
# Une variation sur le bayésien naïf, censé donner de meilleurs résultats
from sklearn.naive_bayes import ComplementNB
CN_classifier = ComplementNB()
CN_classifier.fit(X_entraine,y_entraine)
y_teste_predit = CN_classifier.predict(X_teste)

In [ ]:
# 7.q
CNB_exactitude, CNB_macro_precision, CNB_macro_rappel, CNB_macro_f1 =\
calcule_presente_evaluation(y_teste, y_teste_predit,
                            l_labels, "Complément Bayes Naïf")

               precision    recall  f1-score   support

    Bien-être       0.59      0.37      0.46      1562
Bienveillance       0.36      0.50      0.42       648
     Mal-être       0.70      0.66      0.68      1929
 Malveillance       0.40      0.52      0.45       560
       Neutre       0.09      0.25      0.14       166

     accuracy                           0.52      4865
    macro avg       0.43      0.46      0.43      4865
 weighted avg       0.56      0.52      0.53      4865



###### Régression logistique (multinomiale)
<p>
hyper-paramètres à faire éventuellement varier :<p>
- multi_class : 'auto' = 'multinomial' si non binaire ; 'ovr'<p>
- penalty : 'l2' (par défaut), 'l1', 'none'

In [ ]:
# 7.r
from sklearn.linear_model import LogisticRegression
LR_classifier = LogisticRegression(multi_class='ovr')
LR_classifier.fit(X_entraine,y_entraine)
y_teste_predit = LR_classifier.predict(X_teste)

In [ ]:
# 7.s
LR_exactitude, LR_macro_precision, LR_macro_rappel, LR_macro_f1 =\
calcule_presente_evaluation(y_teste, y_teste_predit,
                            l_labels, "Régression logistique")
# en gardant le solver par défaut, régularisation l2 donne de meilleurs résultats que none (l1 indisponible)
# et multi_class One vs Rest est un peu meilleur que multinomial

               precision    recall  f1-score   support

    Bien-être       0.58      0.60      0.59      1590
Bienveillance       0.41      0.46      0.43       644
     Mal-être       0.73      0.71      0.72      1879
 Malveillance       0.55      0.47      0.51       602
       Neutre       0.23      0.25      0.24       150

     accuracy                           0.60      4865
    macro avg       0.50      0.50      0.50      4865
 weighted avg       0.60      0.60      0.60      4865



###### Support Vector Machine
<p>
hyper-paramètres à faire éventuellement varier :<p>
- kernel : 'rbf' est le défaut, polynomial une alternative 'poly' (degree = 3, cubique est alors le défaut)<p>
- C : inverse de la force de la pénalité L2, 1.0 est le défaut<p>
- gamma : 'scale' (défaut, 1/(V * variance) ) ou 'auto' (1/V),
    
###### Plus long à exécuter que les autres méthodes, les paramètres par défaut semblent être les meilleurs

In [ ]:
# 7.t
# SVM Classifier
from sklearn.svm import SVC, LinearSVC, NuSVC
SVM_classifier = SVC()
SVM_classifier.fit(X_entraine,y_entraine)
y_teste_predit = SVM_classifier.predict(X_teste)

In [ ]:
# 7.u
titre = "Support Vector Machine - RBF"
SVC_exactitude, SVC_macro_precision, SVC_macro_rappel, SVC_macro_f1 =\
calcule_presente_evaluation(y_teste, y_teste_predit,
                            l_labels, titre)
# les paramètres par défaut sont un peu moins bons qu'avec la régression logistique
# polynomial est moins bon que RBF

               precision    recall  f1-score   support

    Bien-être       0.56      0.66      0.60      1590
Bienveillance       0.42      0.46      0.44       644
     Mal-être       0.73      0.71      0.72      1879
 Malveillance       0.59      0.43      0.50       602
       Neutre       0.49      0.13      0.20       150

     accuracy                           0.60      4865
    macro avg       0.56      0.48      0.49      4865
 weighted avg       0.61      0.60      0.60      4865



##### On essaye aussi XGBoost
Nécessite un réglage fin de nombreux paramètres que nous ne ferons pas ici, et qui pourraient donner de meilleurs résultats

In [ ]:
# 7.v
# XGBoost Classifier
from xgboost import XGBClassifier
XGB_classifier = XGBClassifier()
XGB_classifier.fit(X_entraine,y_entraine)
y_teste_predit = XGB_classifier.predict(X_teste)

In [ ]:
# 7.w
titre = "XGBoost"
XGB_exactitude, XGB_macro_precision, XGB_macro_rappel, XGB_macro_f1 =\
calcule_presente_evaluation(y_teste, y_teste_predit,
                            l_labels, titre)
# avec le paramétrage par défaut, on est moins bon que régression linéaire, ou SVM
# mais beaucoup d'hyper-paramètres à régler...

               precision    recall  f1-score   support

    Bien-être       0.50      0.75      0.60      1590
Bienveillance       0.43      0.30      0.35       644
     Mal-être       0.71      0.59      0.64      1879
 Malveillance       0.56      0.35      0.43       602
       Neutre       0.21      0.17      0.19       150

     accuracy                           0.56      4865
    macro avg       0.48      0.43      0.44      4865
 weighted avg       0.57      0.56      0.55      4865



##### Vectorisation doc2vec

In [ ]:
# 8.a
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec

In [ ]:
# 8.b
# gensim nécessite une liste de liste de tokens et non de textes
# un split simple est suffisant pour ce texte déjà prétraite
l_textes_l_tokens = [texte.split() for texte in l_textes]

# et des documents taggés
l_docs = [TaggedDocument(doc, [i]) for i, doc in enumerate(l_textes_l_tokens)]

##### Les principaux paramètres du modèle et de son entraînement :
###### vector_size : nombre de dimensions de l'espace d'embeddings, défaut à 100
###### window : taille de la fenêtre d'examen des cooccurrences (plus précisément distance max entre terme predit et terme courant centre), défaut à 5
###### dm : 0 pour DBOW, 1 pour distributed memory (le défaut)
###### min_count : fréquence absolue minimum dans le corpus pour qu'un terme soit retenu.
###### workers : nb d'unités participant au processing (quand la machine le permet), défaut = 3
###### epochs : le nombre de périodes d'entraînement (itérations), à fixer explicitement. 10 est un nombre généralement retenu, pour des petits corpus, cela coûte moins cher, et il est intéressant de poursuivre l'entraînement plus loin

##### On essaye d'abord avec l'algorithme doc2vec par défaut

In [ ]:
# 8.c
# on va au nombre de dimensions habituellement retenu
NB_DIMS_DOC2VEC = 300

In [ ]:
# 8.d
# on pourrait donner d'autres valeurs aux paramètres, et en faire une fonction
modele_doc2vec = Doc2Vec(l_docs, dm=1, vector_size=NB_DIMS_DOC2VEC, window=6, min_count=2, workers=4, epochs=32)

In [ ]:
# 8.e
# créer les entrées vectorisées par doc2vec pour les splits entraînement et test
# ainsi que les entrées labels
# on va reprendre notations classiques

# on aurait pu en faire une fonction

# textes de départ
X_textes_entraine = df_entraine_equilibre['commentaire_avec_bigrammes'].tolist()
X_textes_entraine = [str(texte).split() for texte in X_textes_entraine]
X_textes_teste = df_teste['commentaire_avec_bigrammes'].tolist()
X_textes_teste = [str(texte).split() for texte in X_textes_teste]

# transformés par doc2vec
X_entraine = np.zeros([len(X_textes_entraine), NB_DIMS_DOC2VEC])
for ix, l_tokens in enumerate(X_textes_entraine):
    v_texte = modele_doc2vec.infer_vector(l_tokens, epochs=32)
    X_entraine[ix] = v_texte

X_teste = np.zeros([len(X_textes_teste), NB_DIMS_DOC2VEC])
for ix, l_tokens in enumerate(X_textes_teste):
    v_texte = modele_doc2vec.infer_vector(l_tokens, epochs=32)
    X_teste[ix] = v_texte

# labels (en fait, pas de changements)
y_entraine = df_entraine_equilibre['émotion'].tolist()
y_teste = df_teste['émotion'].tolist()

In [ ]:
# 8.f
# Naïve Bayes a besoin de valeurs positives en entrée, ne marche pas
# on regarde directement avec la régression logistique

VLR_classifier = LogisticRegression(multi_class='ovr')
VLR_classifier.fit(X_entraine,y_entraine)
y_teste_predit = VLR_classifier.predict(X_teste)

VLR_exactitude, VLR_macro_precision, VLR_macro_rappel, VLR_macro_f1 =\
calcule_presente_evaluation(y_teste, y_teste_predit,
                            l_labels, "doc2vec - Régression logistique")
# les résultats ne sont pas si bons...

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.7/dist-packages/sklearn/lin

               precision    recall  f1-score   support

    Bien-être       0.45      0.46      0.45      1562
Bienveillance       0.27      0.20      0.23       648
     Mal-être       0.49      0.54      0.51      1929
 Malveillance       0.35      0.31      0.33       560
       Neutre       0.14      0.14      0.14       166

     accuracy                           0.43      4865
    macro avg       0.34      0.33      0.33      4865
 weighted avg       0.42      0.43      0.42      4865



In [ ]:
# 8.g
# autre essai avec SVM
VSVM_classifier = SVC()
VSVM_classifier.fit(X_entraine,y_entraine)
y_teste_predit = VSVM_classifier.predict(X_teste)

titre = "doc2vec - Support Vector Machine - RBF"
VSVC_exactitude, VSVC_macro_precision, VSVC_macro_rappel, VSVC_macro_f1 =\
calcule_presente_evaluation(y_teste, y_teste_predit,
                            l_labels, titre)
# mieux que la régression linéaire sous doc2vec, mais moins bon qu'avec tf-idf

               precision    recall  f1-score   support

    Bien-être       0.46      0.60      0.52      1562
Bienveillance       0.32      0.40      0.35       648
     Mal-être       0.66      0.53      0.59      1929
 Malveillance       0.34      0.26      0.29       560
       Neutre       0.28      0.08      0.12       166

     accuracy                           0.49      4865
    macro avg       0.41      0.37      0.38      4865
 weighted avg       0.50      0.49      0.49      4865



In [ ]:
# 8.h
# une possibité est de combiner les deux algorithmes doc2vec par concaténation,
# et en rajoutant quelques paramètres techniques

# DBOW : avec negative sampling (très technique...)
doc2vec_dbow = Doc2Vec(l_docs, dm=0, vector_size=NB_DIMS_DOC2VEC, negative=5, hs=0, window=6, min_count=2, workers=4, epochs=32)
# DMM, avec dm_mean=1
doc2vec_dmm = Doc2Vec(l_docs, dm=1, vector_size=NB_DIMS_DOC2VEC, dm_mean=1, window=6, min_count=2, workers=4, epochs=32)

In [ ]:
# 8.i
# concaténation des deux algorithmes
doc2vec_dbow_dmm = ConcatenatedDoc2Vec([doc2vec_dbow, doc2vec_dmm])

In [ ]:
# 8.j
# on a bien en sortie une nouvelle dimension de 300 + 300
x = doc2vec_dbow_dmm.infer_vector(['juste, un, essai'])
print(len(x))

600


In [ ]:
# 8.k
# recréation des jeux de test et d'entrainement (uniquement ce qui a été changé)
X_entraine = np.zeros([len(X_textes_entraine), 2 * NB_DIMS_DOC2VEC])
for ix, l_tokens in enumerate(X_textes_entraine):
    v_texte = doc2vec_dbow_dmm.infer_vector(l_tokens, epochs=32)
    X_entraine[ix] = v_texte

X_teste = np.zeros([len(X_textes_teste), 2 * NB_DIMS_DOC2VEC])
for ix, l_tokens in enumerate(X_textes_teste):
    v_texte = doc2vec_dbow_dmm.infer_vector(l_tokens, epochs=32)
    X_teste[ix] = v_texte


In [ ]:
# 8.l
# on réessaye avec SVM (prend 3 MINUTES)
VSVM_classifier = SVC()
VSVM_classifier.fit(X_entraine,y_entraine)
y_teste_predit = VSVM_classifier.predict(X_teste)

titre = "doc2vec * 2 - Support Vector Machine - RBF"
VSVC_exactitude, VSVC_macro_precision, VSVC_macro_rappel, VSVC_macro_f1 =\
calcule_presente_evaluation(y_teste, y_teste_predit,
                            l_labels, titre)
# meilleur qu'avec un seul doc2vec, mais toujours moin bon qu'avec td-idf

               precision    recall  f1-score   support

    Bien-être       0.50      0.60      0.55      1562
Bienveillance       0.37      0.45      0.41       648
     Mal-être       0.69      0.58      0.63      1929
 Malveillance       0.37      0.32      0.34       560
       Neutre       0.28      0.13      0.18       166

     accuracy                           0.53      4865
    macro avg       0.44      0.42      0.42      4865
 weighted avg       0.53      0.53      0.53      4865



### Utilisation de BERT

##### Il faut d'abord mettre les données sous format HuggingFace, et donc créer un dataset HuggingFace à partir des données d'entraînement rééquilibrées auparavant, et des données de test, à distinguer
- Je n'ai pas trouvé comment charger des objets pandas dans un même dataset en distinguant 'train' et 'test', je vais passer par des .csv

###### Par ailleurs il faut que la colonne des labels soit sous forme d'index numérique, entre 0 et N-1, N étant le nombre de classes

In [1]:
# 9.a
# la liste des labels doit être une vraie liste (sinon non reconnaissable dans ClassLabel)
l_labels = list(l_labels)

NameError: ignored

In [ ]:
# 9.b
# passage de libellé à numéro de label et inversement (mais dans les datasets on utilisera ClassLabel.int2str et str2int)
d_int2str_emotions = {ix: label for ix, label in enumerate(l_labels)}
print(d_int2str_emotions)
d_str2int_emotions = {label: ix for ix, label in enumerate(l_labels)}
print(d_str2int_emotions)

{0: 'Bien-être', 1: 'Bienveillance', 2: 'Mal-être', 3: 'Malveillance', 4: 'Neutre'}
{'Bien-être': 0, 'Bienveillance': 1, 'Mal-être': 2, 'Malveillance': 3, 'Neutre': 4}


In [ ]:
# 9.c
# comme lecture brute, il faut aussi remplacer impérativement les nouvelles lignes, 
# ainsi qu'éventuellement ; et " pour être bien sûr, ce qui permet de ne plus générer la quotation totale
def remove_sep_characters(s):
  s = str(s)
  s = s.replace(';', ' ').replace('"', ' ').replace('\n', ' ').replace('\r', ' ')
  return s

In [ ]:
# 9.d
# transformation du jeu d'entraînement équilibré en fichier .csv
df_emotions_entraine = df_entraine_equilibre[['commentaire_ss_emojis', 'émotion']]
# il semble qu'il vaille mieux avoir comme nom 'label' pour la colonne des labels
df_emotions_entraine['label'] = df_emotions_entraine['émotion'].progress_apply(lambda x: d_str2int_emotions[x])
df_emotions_entraine['commentaire_ss_emojis'] = df_emotions_entraine['commentaire_ss_emojis'].progress_map(lambda x: remove_sep_characters(x))
df_emotions_entraine = df_emotions_entraine[['commentaire_ss_emojis', 'label']]
nom_chemin_fichier_entraine = "/content/drive/My Drive/Datasets NLP/ds_emotions_entraine.csv"
df_emotions_entraine.to_csv(nom_chemin_fichier_entraine, sep=';', encoding='utf-8', 
                            header=True, index=False)
#files.download(nom_chemin_fichier_entraine)

100%|██████████| 13490/13490 [00:00<00:00, 600171.42it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
100%|██████████| 13490/13490 [00:00<00:00, 372163.68it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# 9.e
# et transformation du jeu de test en fichier .csv
df_emotions_teste = df_teste[['commentaire_ss_emojis', 'émotion']]
df_emotions_teste['label'] = df_emotions_teste['émotion'].progress_apply(lambda x: d_str2int_emotions[x])
df_emotions_teste['commentaire_ss_emojis'] = df_emotions_teste['commentaire_ss_emojis'].progress_map(lambda x: remove_sep_characters(x))
df_emotions_teste = df_emotions_teste[['commentaire_ss_emojis', 'label']]
nom_chemin_fichier_teste = "/content/drive/My Drive/Datasets NLP/ds_emotions_teste.csv"
df_emotions_teste.to_csv(nom_chemin_fichier_teste, sep=';', encoding='utf-8', 
                         header=True, index=False)
#files.download(nom_chemin_fichier_teste)

100%|██████████| 4865/4865 [00:00<00:00, 509253.76it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
100%|██████████| 4865/4865 [00:00<00:00, 374004.09it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 9.f
# imports nécessaires pour la partie Dataset
from datasets import Dataset
from datasets import load_dataset
from datasets import Features, Value
from datasets.features import ClassLabel

In [ ]:
# 9.g
# attention ! il faut transmettre les arguments corrects, correspondant au read_csv de pandas, ici sep=';', encoding='utf-8'
ds_emotions = load_dataset('csv', data_files={'train': nom_chemin_fichier_entraine, 'test': nom_chemin_fichier_teste}, sep=';', encoding='utf-8',
                           features=Features({'commentaire_ss_emojis': Value(dtype='string', id=None), 
                                              'label': ClassLabel(num_classes=5, names=l_labels, names_file=None, id=None)}))

Using custom data configuration default-e61aa57bb2ee7bde


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-e61aa57bb2ee7bde/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# 9.h
# vérification du chargement correct des données
ds_emotions

DatasetDict({
    train: Dataset({
        features: ['commentaire_ss_emojis', 'label'],
        num_rows: 13490
    })
    test: Dataset({
        features: ['commentaire_ss_emojis', 'label'],
        num_rows: 4865
    })
})

In [ ]:
# 9.h.bis
ds_emotions['train'].features

{'commentaire_ss_emojis': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=5, names=['Bien-être', 'Bienveillance', 'Mal-être', 'Malveillance', 'Neutre'], names_file=None, id=None)}

### Chargement du modèle pré-entraîné Flaubert

In [2]:
# 9.i
nom_modele = 'flaubert/flaubert_base_cased' 

In [3]:
# 9.j
from transformers import AutoTokenizer

In [4]:
# 9.k
# pour ne pas se poser la question des commentaires trop long, simplement tronquer la fin de ces commentaires
tokeniseur = AutoTokenizer.from_pretrained(nom_modele, use_fast=True, truncation=True, padding=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.49M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/875k [00:00<?, ?B/s]

In [5]:
# 9.l
# examiner caractères spéciaux
print(tokeniseur.special_tokens_map)
# le github de Flaubert indique :
# The BERT [CLS] token correspond to the first hidden state of the last layer
# cls_embedding = last_layer[:, 0, :]
# semble en contradiction avec ce que renvoie la map 

# confirmer qu'on n'a pas droit à 512 tokens (on tronque à 512)
print(tokeniseur.model_max_length)

{'bos_token': '<s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '</s>', 'mask_token': '<special1>', 'additional_special_tokens': ['<special0>', '<special1>', '<special2>', '<special3>', '<special4>', '<special5>', '<special6>', '<special7>', '<special8>', '<special9>']}
512


In [8]:
texte = "essai de tokénisation du dérèglement climatique"
texte_encode = tokeniseur.encode(texte)
print(texte_encode)
l_tokens_texte = tokeniseur.convert_ids_to_tokens(texte_encode)
print(l_tokens_texte)

[0, 3034, 15, 1454, 39292, 2728, 28, 10693, 19765, 3900, 5483, 1]
['<s>', 'essai</w>', 'de</w>', 'to', 'kén', 'isation</w>', 'du</w>', 'dér', 'èg', 'lement</w>', 'climatique</w>', '</s>']


In [9]:
tokeniseur.add_tokens(['tokénisation', 'dérèglement'])

2

In [10]:
texte = "essai de tokénisation du dérèglement climatique"
texte_encode = tokeniseur.encode(texte)
print(texte_encode)
l_tokens_texte = tokeniseur.convert_ids_to_tokens(texte_encode)
print(l_tokens_texte)

[0, 3034, 15, 68729, 28, 68730, 5483, 1]
['<s>', 'essai</w>', 'de</w>', 'tokénisation', 'du</w>', 'dérèglement', 'climatique</w>', '</s>']


In [ ]:
# 9.m
from transformers import AutoModel

In [ ]:
# 9.n
import torch

In [ ]:
# 9.o
# être sûr d'utiliser la GPU si elle est disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# avoir accès au cas où, à toutes les couches cachées
modele = AutoModel.from_pretrained(nom_modele, output_hidden_states = True).to(device)

Downloading:   0%|          | 0.00/528M [00:00<?, ?B/s]

Some weights of the model checkpoint at flaubert/flaubert_base_cased were not used when initializing FlaubertModel: ['pred_layer.proj.bias', 'pred_layer.proj.weight']
- This IS expected if you are initializing FlaubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# 9.i
# essai sur un seul commentaire
commentaire = df_emotions_entraine.iloc[14]['commentaire_ss_emojis']
print(commentaire)

Et aller c'est encore et toujours de la faute au réchauffement climatique...🌍c'est vraiment pas de chance hein... Alors que le sable est la matière première la plus utilisée dans le monde !!


In [ ]:
# 9.j
# liste des tokens
commentaire_encode = tokeniseur.encode(commentaire)
print(commentaire_encode)
l_tokens_commentaire = tokeniseur.convert_ids_to_tokens(commentaire_encode)
print(l_tokens_commentaire)
print(len(l_tokens_commentaire))
# on perd beaucoup !!

# sous forme de tenseur PyTorch (pt)
commentaire_tokenise_pt = tokeniseur.encode(commentaire, return_tensors="pt").to(device)
print(commentaire_tokenise_pt.shape)

[0, 133, 490, 79, 27, 117, 18, 155, 15, 17, 2462, 36, 9972, 5483, 75, 3, 79, 27, 413, 42, 15, 1489, 9390, 75, 646, 32, 20, 4066, 27, 17, 479, 179, 17, 44, 2634, 33, 20, 144, 82, 82, 1]
['<s>', 'Et</w>', 'aller</w>', "c'</w>", 'est</w>', 'encore</w>', 'et</w>', 'toujours</w>', 'de</w>', 'la</w>', 'faute</w>', 'au</w>', 'réchauffement</w>', 'climatique</w>', '...</w>', '<unk>', "c'</w>", 'est</w>', 'vraiment</w>', 'pas</w>', 'de</w>', 'chance</w>', 'hein</w>', '...</w>', 'Alors</w>', 'que</w>', 'le</w>', 'sable</w>', 'est</w>', 'la</w>', 'matière</w>', 'première</w>', 'la</w>', 'plus</w>', 'utilisée</w>', 'dans</w>', 'le</w>', 'monde</w>', '!</w>', '!</w>', '</s>']
41
torch.Size([1, 41])


In [ ]:
# 9.k
# mettre en entrée le texte tokenizé, et récupérer la sortie
sorties = modele(commentaire_tokenise_pt)

In [ ]:
# 9.l
# vérifier contenu de la sortie des transformeurs
sorties_transformers = sorties['hidden_states']
print(type(sorties_transformers))
print(len(sorties_transformers))
for l in range(len(sorties_transformers)):
  print(type(sorties_transformers[l]))
  print(l, sorties_transformers[l].shape)

<class 'tuple'>
13
<class 'torch.Tensor'>
0 torch.Size([1, 41, 768])
<class 'torch.Tensor'>
1 torch.Size([1, 41, 768])
<class 'torch.Tensor'>
2 torch.Size([1, 41, 768])
<class 'torch.Tensor'>
3 torch.Size([1, 41, 768])
<class 'torch.Tensor'>
4 torch.Size([1, 41, 768])
<class 'torch.Tensor'>
5 torch.Size([1, 41, 768])
<class 'torch.Tensor'>
6 torch.Size([1, 41, 768])
<class 'torch.Tensor'>
7 torch.Size([1, 41, 768])
<class 'torch.Tensor'>
8 torch.Size([1, 41, 768])
<class 'torch.Tensor'>
9 torch.Size([1, 41, 768])
<class 'torch.Tensor'>
10 torch.Size([1, 41, 768])
<class 'torch.Tensor'>
11 torch.Size([1, 41, 768])
<class 'torch.Tensor'>
12 torch.Size([1, 41, 768])


In [ ]:
# 9.l.bis
print(sorties.last_hidden_state.shape)

torch.Size([1, 41, 768])


In [ ]:
# 9.l.ter
# les 10 premières composantes du vecteur CLS de la dernière couche cachée, à partir de laquelle on pourra faire la classification
print(sorties.last_hidden_state[0, 0, 0:10])
print(sorties_transformers[12][0, 0, 0:10])

tensor([ 0.0289, -1.1699,  0.7259, -0.0253, -0.9903, -0.1525,  0.5255,  0.0471,
        -0.4575, -1.1282], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([ 0.0289, -1.1699,  0.7259, -0.0253, -0.9903, -0.1525,  0.5255,  0.0471,
        -0.4575, -1.1282], device='cuda:0', grad_fn=<SliceBackward0>)


#### utilisation de la fonction DataSet.map() pour :
- tokéniser l'ensemble des commentaires des deux splits, entraînement et test
- extraire la dernière couche du modèle pour l'ensemble des commentaires

In [ ]:
# 9.m
# batched = True, on va mettre les commentaires en batch
# batch_size = None, on prend tout d'un coup, et la longueur prise par les masques d'attention est celle fixée par le masque le plus long
# on traite d'un coup les deux splits d'entraînement et de test
# on doit spécifier truncation=True pour s'assurer que les commentaires soient correctement tronqués
commentaires_tokenises = ds_emotions.map(lambda x: tokeniseur(x['commentaire_ss_emojis'], truncation=True, padding=True), batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# 9.n
# on a maintenant les tokénisations de tpous les éléments du dataset, entraînement et test
commentaires_tokenises

DatasetDict({
    train: Dataset({
        features: ['commentaire_ss_emojis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 13490
    })
    test: Dataset({
        features: ['commentaire_ss_emojis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4865
    })
})

In [ ]:
# 9.n.bis
print(len(commentaires_tokenises['train'][0]['input_ids']))
print(commentaires_tokenises['train'][0]['input_ids'])
print(commentaires_tokenises['train'][0]['attention_mask'])

512
[0, 128, 32, 7151, 22, 16509, 515, 8016, 3914, 43, 89, 89, 15609, 1196, 324, 2441, 89, 190, 624, 3277, 26282, 704, 312, 31127, 194, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [ ]:
# 9.n.ter
print(type(commentaires_tokenises['train'][0]['input_ids']))

<class 'list'>


In [ ]:
# 9.o
# utiliser ces résultats en entrée de la fonction d'exploitation du modèle
# on enrichit le batch avec une nouvelle colonne représentant la moyenne des 
# derniers états cachés
def obtention_derniere_couche_cachee(batch):

  input_ids = torch.tensor(batch["input_ids"]).to(device)  
  attention_mask = torch.tensor(batch["attention_mask"]).to(device)

  # pas de calcul de gradients au niveau du tenseur pyTorch
  with torch.no_grad():
    derniere_couche_cachee = modele(input_ids, attention_mask).last_hidden_state
    derniere_couche_cachee = derniere_couche_cachee.cpu().numpy()

  # comme on n'est pas dans le cadre de l'utilisation classique avec tête de classification
  # et utilisation du token CLS, on va simplement prendre la moyenne des tokens (significatifs)
  # de la dernière couche

  dims_dcc = derniere_couche_cachee.shape
  # ne prendre en compte que les valeurs ou le masque d'attention est à 1
  masque_bool = ~np.array(batch["attention_mask"]).astype(bool)
  masque_bool = np.repeat(masque_bool, dims_dcc[-1], axis=-1)
  masque_bool = masque_bool.reshape(dims_dcc)
  moyenne_avec_masque = np.ma.array(derniere_couche_cachee, mask=masque_bool).mean(axis=1)
  batch["état_caché"] = moyenne_avec_masque.data
  return batch


###### Il faut avoir mis le mode d'exécution en GPU, sinon au lieu de 10 minutes, on a 10 heures d'exécution...

In [ ]:
# 9.p
commentaires_encodes = commentaires_tokenises.map(obtention_derniere_couche_cachee, batched=True, batch_size=16)

  0%|          | 0/844 [00:00<?, ?ba/s]

  0%|          | 0/305 [00:00<?, ?ba/s]

In [ ]:
# 9.q
commentaires_encodes

DatasetDict({
    train: Dataset({
        features: ['commentaire_ss_emojis', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'état_caché'],
        num_rows: 13490
    })
    test: Dataset({
        features: ['commentaire_ss_emojis', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'état_caché'],
        num_rows: 4865
    })
})

In [ ]:
# 9.q.bis
commentaires_encodes['train'].features

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'commentaire_ss_emojis': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'label': ClassLabel(num_classes=5, names=['Bien-être', 'Bienveillance', 'Mal-être', 'Malveillance', 'Neutre'], names_file=None, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'état_caché': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None)}

In [ ]:
# 9.r
# éléments d'entraînement, et de test
X_entraine = np.array(commentaires_encodes["train"]["état_caché"])
X_teste = np.array(commentaires_encodes["test"]["état_caché"])
y_entraine = np.array(commentaires_encodes["train"]["label"])
y_teste = np.array(commentaires_encodes["test"]["label"])


In [ ]:
# 9.s
# vérifier les dimensions des différents éléments
print(X_entraine.shape, X_teste.shape)
print(y_entraine.shape, y_teste.shape)

(13490, 768) (4865, 768)
(13490,) (4865,)


###### si reprise, suite à changement de mode d'exécution
###### et il faut être repassé dans les fonctions de calcul et de présentation des métrics 

In [ ]:
# 9.t
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [ ]:
# 9.u
BLR_classifier = LogisticRegression(penalty='none', max_iter=200)
BLR_classifier.fit(X_entraine,y_entraine)
y_teste_predit = BLR_classifier.predict(X_teste)

BLR_exactitude, BLR_macro_precision, BLR_macro_rappel, BLR_macro_f1 =\
calcule_presente_evaluation(y_teste, y_teste_predit,
                            l_labels, "FlauBERT brut - Régression logistique")
# n'est pas très bon

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



               precision    recall  f1-score   support

    Bien-être       0.43      0.39      0.41      1503
Bienveillance       0.33      0.40      0.36       660
     Mal-être       0.61      0.52      0.56      1941
 Malveillance       0.22      0.23      0.23       602
       Neutre       0.09      0.23      0.13       159

     accuracy                           0.42      4865
    macro avg       0.34      0.36      0.34      4865
 weighted avg       0.45      0.42      0.43      4865



In [ ]:
# 9.v
BSVM_classifier = SVC()
BSVM_classifier.fit(X_entraine,y_entraine)
y_teste_predit = BSVM_classifier.predict(X_teste)

BSVM_exactitude, BSVM_macro_precision, BSVM_macro_rappel, BSVM_macro_f1 =\
calcule_presente_evaluation(y_teste, y_teste_predit,
                            l_labels, "FlauBERT brut - SVM")
# n'est pas non plus très bon

               precision    recall  f1-score   support

    Bien-être       0.43      0.51      0.47      1503
Bienveillance       0.33      0.39      0.36       660
     Mal-être       0.60      0.63      0.61      1941
 Malveillance       0.30      0.09      0.14       602
       Neutre       0.20      0.11      0.14       159

     accuracy                           0.48      4865
    macro avg       0.37      0.35      0.34      4865
 weighted avg       0.46      0.48      0.46      4865



#### Réglage fin d'un transformer à fins de classification

In [ ]:
# 10.a
from transformers import AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments

In [ ]:
# 10.b
# Chargement du modèle pré-entraîné
nb_labels = len(l_labels)
modele = AutoModelForSequenceClassification.from_pretrained(nom_modele, num_labels=nb_labels).to(device)

Some weights of the model checkpoint at flaubert/flaubert_base_cased were not used when initializing FlaubertForSequenceClassification: ['pred_layer.proj.bias', 'pred_layer.proj.weight']
- This IS expected if you are initializing FlaubertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaubertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of FlaubertForSequenceClassification were not initialized from the model checkpoint at flaubert/flaubert_base_cased and are newly initialized: ['sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for pre

In [ ]:
# 10.c
# il faut aussi ne garder que les colonnes intéressantes et les mettre au format tenseur pyTorch
commentaires_encodes.remove_columns(["commentaire_ss_emojis", "état_caché"])
commentaires_encodes.set_format("torch", columns=["input_ids", "attention_mask", "label"])


In [ ]:
# 10.d
# arguments pour entraîner le modèle
taille_batch = 16
log_tous_les = len(commentaires_encodes["train"]) // taille_batch
args_training = TrainingArguments(output_dir="results",
                                  num_train_epochs=2, # 3 valeur par défaut
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=taille_batch,
                                  per_device_eval_batch_size=taille_batch,
                                  metric_for_best_model="f1",
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=log_tous_les,)

In [ ]:
# 10.e
# il faut aussi passer la fonction d'évaluation avec la métrique f1
def calcule_metriques(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = metrics.f1_score(labels, preds, average="macro")
  acc = metrics.accuracy_score(labels, preds)
  return {"accuracy": acc, "f1": f1}


In [ ]:
# 10.f
entraineur = Trainer(model=modele, args=args_training,
                     compute_metrics=calcule_metriques,
                     tokenizer=tokeniseur,
                     train_dataset=commentaires_encodes["train"],
                     eval_dataset=commentaires_encodes["test"])

In [ ]:
# 10.g
# problèmes de mémoire...
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    1082 MB |    1801 MB |   12694 GB |   12693 GB |\n|       from large pool |    1056 MB |    1776 MB |   12691 GB |   12690 GB |\n|       from small pool |      26 MB |      26 MB |       2 GB |       2 GB |\n|---------------------------------------------------------------------------|\n| Active memory         |    1082 MB |    1801 MB |   12694 GB |   12693 GB |\n|       from large pool |    1056 MB |    1776 MB |

###### maintenant, entraînement réel du modèle
- Je n'ai pu encore franchir cette étape
- problème de mémoire (mais peut-être résolu par transformation du format en tenseur et élimination colonnes inutiles)
- problème de key 'Loss' manquante (mais peut-être résolu en renommant la colonne des labels 'label')

In [ ]:
# 10.h
# entraînement !!!
entraineur.train()

The following columns in the training set  don't have a corresponding argument in `FlaubertForSequenceClassification.forward` and have been ignored: état_caché, commentaire_ss_emojis.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 13490
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1688


RuntimeError: ignored

### Annexe : ouverture de quelques datasets Hugging Face en français

In [ ]:
from datasets import Dataset
from datasets import load_dataset

In [ ]:
ds_allocine = load_dataset("allocine")

Downloading:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/866 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/66.6M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset allocine_dataset downloaded and prepared to /root/.cache/huggingface/datasets/allocine_dataset/allocine/1.0.0/91f700d606838c22c5c370846746e60503219d0c1f16ed96bfd1fa19a73458eb. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
ds_allocine

DatasetDict({
    train: Dataset({
        features: ['review', 'label'],
        num_rows: 160000
    })
    validation: Dataset({
        features: ['review', 'label'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['review', 'label'],
        num_rows: 20000
    })
})

In [ ]:
ds_amazon_reviews_multi = load_dataset("amazon_reviews_multi")

No config specified, defaulting to: amazon_reviews_multi/all_languages


  0%|          | 0/6 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/90.3M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/82.0M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/77.5M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset amazon_reviews_multi downloaded and prepared to /root/.cache/huggingface/datasets/amazon_reviews_multi/all_languages/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]